In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import permutations 
import pickle

import aux_functions as af

[20:00:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [2]:
#load xgb model
# model = pickle.load(open('xgb_model.sav', 'rb'))

In [2]:
#load test data
taxi_test = pd.read_csv('data/test.csv')

In [3]:
taxi_test

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N
...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N


I am using the testing dataset to randomly generate pickup locations for testing

In [70]:
np.random.seed(seed=4321)
test_set_idx = np.random.choice(len(taxi_test),5)
test_set = taxi_test.iloc[test_set_idx].reset_index()[['pickup_longitude', 'pickup_latitude']]

#Sample date
test_date = datetime(2020, 5, 17, 17)

In [71]:
test_set

,pickup_longitude,pickup_latitude
0,-73.993599,40.747028
1,-73.863625,40.769852
2,-73.990532,40.751259
3,-73.991287,40.750370
4,-74.014709,40.714329


In [72]:
perm = list(permutations(list(test_set.index))) 

In [73]:
test_full = af.rank(perm, test_set, test_date)

In [74]:
test_full_ranked = test_full.sort_values(by=['rank'], ascending=False)

In [75]:
test_full_ranked.iloc[0]

0                4.000000
1                2.000000
2                3.000000
3                0.000000
4                1.000000
total_time    3468.165573
rank           120.000000
chance           0.016529
Name: 112, dtype: float64

In [76]:
test_full[test_full[0] == 0]

,0,1,2,3,4,total_time,rank,chance
0,0,1,2,3,4,5261.886108,66.0,0.009091
1,0,1,2,4,3,6195.810303,8.0,0.001102
2,0,1,3,2,4,5427.394226,59.0,0.008127
3,0,1,3,4,2,6038.931580,17.0,0.002342
4,0,1,4,2,3,5765.104736,35.0,0.004821
5,0,1,4,3,2,5859.303101,30.0,0.004132
6,0,2,1,3,4,5583.440308,49.0,0.006749
7,0,2,1,4,3,6315.844116,3.0,0.000413
8,0,2,3,1,4,5428.521423,58.0,0.007989
9,0,2,3,4,1,3850.531433,112.0,0.015427


In [77]:
def get_optimal_order(input_df, date, init_pop_size, gens):
    #timer
    start_time = datetime.now()
    
    #getting initial population
    init_pop =  af.get_init_pop(len(input_df), init_pop_size)
    
    for i in range(gens - 1):
        #ranking the population
        ranked = af.rank(init_pop, input_df, date)
        
        best = ranked.sort_values(by=['rank'], ascending=False).iloc[0] 
#         max_prob = af.condense(ranked).chance.max()
#         print(i, best)
    
        #creating next generation
        next_gen = af.get_next_gen(ranked)
        init_pop = next_gen
    
    final_gen = af.condense(af.rank(next_gen, input_df, date))
    
    end_time = datetime.now()
    print(end_time - start_time)
    
    return(final_gen.iloc[:3])
    

In [78]:
get_optimal_order(test_set, test_date, 100, 20)

0:02:21.982410


,0,1,2,3,4,total_time,rank,chance
0,4,2,3,0,1,3468.165573,5050.0,1.0
